In [1]:
from git import Repo

!rm -rf repos/chat-app
repo_url = "https://github.com/stephen-osullivan/streamlit-openai-chatbot.git"
repo_local_path = "repos/chat-app"
# Clone the repository
repo = Repo.clone_from(repo_url, repo_local_path)

# Iterate over all commits
for commit in repo.iter_commits():
    print(f"Commit: {commit.hexsha}", f"Author: {commit.author}", f"Date: {commit.committed_datetime}")
    print(f"Message: {commit.message}\n")

Commit: 7fdfcce6c8759d086fd2b6e1b72da2c0fccbaf94 Author: stephen-osullivan Date: 2024-05-02 11:09:24+00:00
Message: checked working with llamacpp


Commit: 6bc6cb57334394ad5367c876a87abf1ec76245ef Author: stephen-osullivan Date: 2024-05-02 10:17:20+00:00
Message: added image to readme, fixed a hardcoded url


Commit: f8d5c28d4ba45b4f51c1381a6387d8fb9c21d71a Author: stephen-osullivan Date: 2024-05-01 16:26:03+00:00
Message: added docker launch command


Commit: 0f19dedaed9994626d76deab206ec3d1664a25bf Author: stephen-osullivan Date: 2024-05-01 16:20:44+00:00
Message: added docker image


Commit: b90470d0de7e26d0c735628429224ba5f8c63801 Author: stephen-osullivan Date: 2024-05-01 16:15:35+00:00
Message: stream output working for ai response


Commit: 0227e8c13e36b568f257bc86704c607ace043cfe Author: stephen-osullivan Date: 2024-05-01 14:59:36+00:00
Message: app working as intended


Commit: 20930ccac745e99a40ad0fb4a2ff773b38820e38 Author: stephen-osullivan Date: 2024-05-01 12:12:59+00:00
M

In [2]:
# Get a specific commit
commit_hash = "7fdfcce6c8759d086fd2b6e1b72da2c0fccbaf94"
commit = repo.commit(commit_hash)

import os
tree = commit.tree

# Traverse the tree and list files with their sizes
for blob in tree.traverse():
    if blob.type == "blob":
        file_path = os.path.join(repo_local_path, blob.path)
        file_size = os.path.getsize(file_path)
        print(f"{blob.path} - {file_size} bytes")

.gitignore - 3087 bytes
Dockerfile - 225 bytes
Makefile - 472 bytes
README.md - 679 bytes
app.py - 4108 bytes
demo.png - 166996 bytes
requirements.txt - 60 bytes


In [3]:
# Clone the repository (or use Repo.init_from_url for a remote repo)
repo_path = repo_local_path
repo = Repo(repo_path)

# Get the current commit
current_commit = repo.commit("HEAD")

diffs = current_commit.diff("0f19dedaed9994626d76deab206ec3d1664a25bf")
# Find all changed files
changed_files = set()
for change_type in ["A", "M", "D"]:  # A: added, M: modified, D: deleted
    for changed_file in diffs.iter_change_type(change_type):
        changed_files.add(changed_file.b_path)

# Print the changed files
print("Changed files since the previous commit:")
for file_path in changed_files:
    full_file_path = os.path.join(repo_local_path, file_path)
    file_size = os.path.getsize(full_file_path)
    print(f'{file_path}, size: {file_size/1024:.2f} KBs')

Changed files since the previous commit:
README.md, size: 0.66 KBs
demo.png, size: 163.08 KBs
Makefile, size: 0.46 KBs
.gitignore, size: 3.01 KBs
app.py, size: 4.01 KBs


### langchain document loader

In [4]:
from langchain_community.document_loaders import TextLoader

# Load the changed files into LangChain documents
documents = []
for file_path in changed_files:
    full_path = os.path.join(repo_path, file_path)
    if (os.path.getsize(full_file_path) < 10 * 1024): # less than 10kb 
        try:
            loader = TextLoader(full_path, autodetect_encoding=True)
            documents.extend(loader.load())
        except:
            print('Failed to decode:', file_path)

# Now you can use the 'documents' list with LangChain
print(f"Loaded {len(documents)} documents from the changed files.")
doc_names = [doc.metadata['source'] for doc in documents]
print(doc_names)
print('Lengths:', [(n, len(doc.page_content)) for n, doc in zip(doc_names, documents)])

Failed to decode: demo.png
Loaded 4 documents from the changed files.
['repos/chat-app/README.md', 'repos/chat-app/Makefile', 'repos/chat-app/.gitignore', 'repos/chat-app/app.py']
Lengths: [('repos/chat-app/README.md', 679), ('repos/chat-app/Makefile', 472), ('repos/chat-app/.gitignore', 3087), ('repos/chat-app/app.py', 4105)]


In [5]:
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_openai.chat_models import ChatOpenAI


load_dotenv
llm = ChatOpenAI(
    temperature=0.3, 
    max_tokens = 1000)

prompt = ChatPromptTemplate.from_messages([
        ("system", 'You are an expert at reading and understanding code repositories.'),
        ("human", "Please could you summarise this file for me:\nfile name {file_name}\n\n file_contents:\n{file_contents}"),
    ])
chain = prompt | llm | StrOutputParser()

doc = documents[0]
chain.invoke(dict(file_name=doc.metadata['source'], file_contents = doc.page_content))



AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-QEDUJ***************************************tgw6. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}